## Import Dependencies

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import re

from sklearn.neighbors import KNeighborsClassifier

pd.set_option('display.max_columns', None)
pd.set_option('mode.chained_assignment', None)

## Data Acquisition

In [2]:
plumbing = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\plumbing_project\plumbing_w_id.csv")

C:\Users\nene0\AppData\Local\Temp\ipykernel_13180\891547134.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  plumbing = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\plumbing_project\plumbing_w_id.csv")


## Data Cleaning

In [3]:
plumbing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34203 entries, 0 to 34202
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer ID                 34203 non-null  int64  
 1   Type                        34203 non-null  object 
 2   Last Job Completed          34203 non-null  object 
 3   Customers Lifetime Revenue  34203 non-null  float64
 4   Lifetime Jobs Completed     34203 non-null  int64  
 5   Lifetime Invoices           34203 non-null  int64  
 6   Invoice #                   34203 non-null  object 
 7   Location City               34203 non-null  object 
 8   Location State              34203 non-null  object 
 9   Location Zip                34203 non-null  float64
 10  Job Type                    34203 non-null  object 
 11  Subtotal                    34203 non-null  object 
 12  Tax                         34203 non-null  object 
 13  Total                       342

In [4]:
plumbing.head()

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Subtotal,Tax,Total,Completion Date,Invoice Date
0,7160966,Commercial,2/2/2024,247901.86,923,982,725617,San Jose,CA,95126.0,Area 1 - Plumbing,$405.00,$0.00000,$405.00,1/11/2023,1/11/2023
1,7160966,Commercial,2/2/2024,247901.86,923,982,725791,San Jose,CA,95116.0,Area 1 - Drain,$115.00,$0.00000,$115.00,12/31/1899,1/4/2023
2,7160966,Commercial,2/2/2024,247901.86,923,982,725796,San Jose,CA,95126.0,Area 1 - Drain,$15.00,$0.00000,$15.00,1/5/2023,1/5/2023
3,7160966,Commercial,2/2/2024,247901.86,923,982,725864,Campbell,CA,95008.0,Area 1 - Drain,$115.00,$0.00000,$115.00,1/25/2023,1/5/2023
4,7160966,Commercial,2/2/2024,247901.86,923,982,725867,San Jose,CA,95112.0,Area 1 - Drain,$115.00,$0.00000,$115.00,1/6/2023,1/6/2023


### Correct Data Types

In [5]:
def clean_finances(data, finance_feature):
    """ This function cleans finance-related features by removing the '$' sign and correcting the data type to float.

    Args:
        data (DataFrame): The DataFrame containing the financial data.
        finance_feature (str): The name of the finance feature to clean and correct the data type.

    Returns:
        pandas.Series: The cleaned feature with data type converted to float."""
    
    data[finance_feature] = data[finance_feature].str.replace('$','')
    data[finance_feature] = data[finance_feature].astype('float64')
    return data[finance_feature]

In [6]:
def correct_datatype(data, feature, datatype):
    """ This function corrects the data type of a selected feature in a DataFrame.

    Args:
        data (DataFrame): The DataFrame containing the feature to be corrected.
        feature (str): The name of the column whose data type needs to be changed.
        datatype (str): The desired data type for the feature (e.g., 'object', 'int32', 'float64').

    Returns:
        pandas.Series: The specified feature having the correct data type."""
    
    data[feature] = data[feature].astype(datatype)
    return data[feature]

In [7]:
plumbing['Subtotal'] = clean_finances(plumbing, 'Subtotal')
plumbing['Tax'] = clean_finances(plumbing, 'Tax')
plumbing['Total'] = clean_finances(plumbing, 'Total')

plumbing['Location Zip'] = correct_datatype(plumbing, 'Location Zip', 'int32') # remove '.0'
plumbing['Location Zip'] = correct_datatype(plumbing, 'Location Zip', 'object')

plumbing['Customer ID'] = correct_datatype(plumbing, 'Customer ID', 'object')

# Add 'Subtotal' and 'Tax' to make sure 'Total' is correct.

plumbing['Total'] = plumbing['Subtotal'] + plumbing['Tax']
plumbing.drop(columns=['Subtotal','Tax'], inplace=True)

In [8]:
plumbing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34203 entries, 0 to 34202
Data columns (total 14 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Customer ID                 34203 non-null  object 
 1   Type                        34203 non-null  object 
 2   Last Job Completed          34203 non-null  object 
 3   Customers Lifetime Revenue  34203 non-null  float64
 4   Lifetime Jobs Completed     34203 non-null  int64  
 5   Lifetime Invoices           34203 non-null  int64  
 6   Invoice #                   34203 non-null  object 
 7   Location City               34203 non-null  object 
 8   Location State              34203 non-null  object 
 9   Location Zip                34203 non-null  object 
 10  Job Type                    34203 non-null  object 
 11  Total                       34203 non-null  float64
 12  Completion Date             34203 non-null  object 
 13  Invoice Date                342

### Duplicated and Missing Invoice #s

In [9]:
plumbing.head()

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date
0,7160966,Commercial,2/2/2024,247901.86,923,982,725617,San Jose,CA,95126,Area 1 - Plumbing,405.0,1/11/2023,1/11/2023
1,7160966,Commercial,2/2/2024,247901.86,923,982,725791,San Jose,CA,95116,Area 1 - Drain,115.0,12/31/1899,1/4/2023
2,7160966,Commercial,2/2/2024,247901.86,923,982,725796,San Jose,CA,95126,Area 1 - Drain,15.0,1/5/2023,1/5/2023
3,7160966,Commercial,2/2/2024,247901.86,923,982,725864,Campbell,CA,95008,Area 1 - Drain,115.0,1/25/2023,1/5/2023
4,7160966,Commercial,2/2/2024,247901.86,923,982,725867,San Jose,CA,95112,Area 1 - Drain,115.0,1/6/2023,1/6/2023


In [10]:
# Remove '-1' and '-2' to aggregate invoices later on.

plumbing['Invoice #'] = plumbing['Invoice #'].str.replace('-1', '')
plumbing['Invoice #'] = plumbing['Invoice #'].str.replace('-2', '')

In [11]:
plumbing['Invoice #'].isna().sum()

1435

In [12]:
def create_new_invoice_id(data, invoice_feature):
    """ This function creates a new DataFrame with the missing invoice IDs filled in. The new invoice IDs will start with '9'.

    Args:
        data (DataFrame): The DataFrame containing the original data.
        invoice_feature (str): The name of the column containing the invoice IDs.

    Returns:
        DataFrame: A new DataFrame with the missing invoice IDs filled in."""
    
    null_id = data[data[invoice_feature].isna()]
    null_id.reset_index(drop=True, inplace=True)

    new_id = pd.DataFrame(np.arange(900000, 900000 + len(null_id)), columns=['New Invoice #'])

    filled_id = null_id.join(new_id)

    filled_id[invoice_feature] = np.where(filled_id[invoice_feature].isna(), filled_id['New Invoice #'], filled_id[invoice_feature])
    filled_id[invoice_feature] = plumbing[invoice_feature].astype('object')

    return filled_id.drop(columns='New Invoice #')

In [13]:
new_ids = create_new_invoice_id(plumbing, 'Invoice #')
new_ids

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date
0,27673217,Residential,1/13/2023,0.0,1,1,725617,San Jose,CA,95128,Area 1 - Drain,215.0,1/13/2023,1/13/2023
1,27673985,Residential,1/13/2023,0.0,1,1,725791,San Jose,CA,95111,Area 1 - Drain,0.0,1/13/2023,1/13/2023
2,27673997,Residential,1/11/2023,0.0,1,1,725796,Sunnyvale,CA,94086,Area 1 - Drain,115.0,1/11/2023,1/11/2023
3,27675009,Residential,1/13/2023,0.0,1,1,725864,San Jose,CA,95118,Area 1 - Drain,115.0,1/13/2023,1/13/2023
4,27582377,Residential,1/11/2023,0.0,1,1,725867,Los Gatos,CA,95030,Area 1 - Drain,115.0,1/11/2023,1/11/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1430,36009903,Residential,12/29/2023,115.0,1,1,701832,Saratoga,CA,95070,Area 1 - Drain,115.0,12/29/2023,12/29/2023
1431,36009939,Residential,12/30/2023,430.0,1,1,701859,San Jose,CA,95122,Area 1 - Plumbing,430.0,12/30/2023,12/30/2023
1432,36010002,Residential,12/28/2023,230.0,1,1,701899,San Jose,CA,95128,Area 1 - Plumbing,230.0,12/28/2023,12/28/2023
1433,36010631,Residential,12/29/2023,430.0,1,1,702024,San Jose,CA,95118,Area 1 - Drain,430.0,12/29/2023,12/28/2023


In [14]:
def fillin_invoice_id(original_df, new_id_df):
    """ This function concatenate original DataFrame with the new IDs DataFrame and returns the full DataFrame with all invoice IDs filled in.

    Args:
        original_df (DataFrame): The original DataFrame containing the data.
        new_id_df (DataFrame): The DataFrame with missing invoice IDs filled in.

    Returns:
        DataFrame: A full DataFrame with all missing invoice IDs filled in."""
    
    # Drop null values.
    original_df.drop(index=original_df[original_df['Invoice #'].isna()].index, inplace=True)

    # Concatenate original dataframe with the id dataframe.
    new_df = pd.concat([original_df, new_id_df], ignore_index=True)
    return new_df

In [15]:
plumbing = fillin_invoice_id(plumbing, new_ids)
plumbing

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date
0,7160966,Commercial,2/2/2024,247901.86,923,982,725617,San Jose,CA,95126,Area 1 - Plumbing,405.0,1/11/2023,1/11/2023
1,7160966,Commercial,2/2/2024,247901.86,923,982,725791,San Jose,CA,95116,Area 1 - Drain,115.0,12/31/1899,1/4/2023
2,7160966,Commercial,2/2/2024,247901.86,923,982,725796,San Jose,CA,95126,Area 1 - Drain,15.0,1/5/2023,1/5/2023
3,7160966,Commercial,2/2/2024,247901.86,923,982,725864,Campbell,CA,95008,Area 1 - Drain,115.0,1/25/2023,1/5/2023
4,7160966,Commercial,2/2/2024,247901.86,923,982,725867,San Jose,CA,95112,Area 1 - Drain,115.0,1/6/2023,1/6/2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34198,36009903,Residential,12/29/2023,115.00,1,1,701832,Saratoga,CA,95070,Area 1 - Drain,115.0,12/29/2023,12/29/2023
34199,36009939,Residential,12/30/2023,430.00,1,1,701859,San Jose,CA,95122,Area 1 - Plumbing,430.0,12/30/2023,12/30/2023
34200,36010002,Residential,12/28/2023,230.00,1,1,701899,San Jose,CA,95128,Area 1 - Plumbing,230.0,12/28/2023,12/28/2023
34201,36010631,Residential,12/29/2023,430.00,1,1,702024,San Jose,CA,95118,Area 1 - Drain,430.0,12/29/2023,12/28/2023


In [16]:
def remove_duplicated_id(data, invoice_feature):
    """ This function removes duplicated rows from the provided DataFrame. Duplicated rows typically belong to invoices marked with '-1' or '-2', and their totals are aggregated to reflect the correct final total.

    Args:
        data (DataFrame): The DataFrame to remove duplicated rows from.
        invoice_feature (str): The name of the invoice ID feature in the DataFrame.

    Returns:
        DataFrame: A DataFrame with duplicated rows removed."""
    
    duplicated_invoices = data[data.duplicated(subset=[invoice_feature], keep=False)]
    removed_duplicated = data.drop(index=data[data.duplicated(subset=invoice_feature, keep=False)].index)

    # Aggregate the duplicated invoice's total.
    agg_total = duplicated_invoices.groupby(invoice_feature)['Total'].sum().reset_index()

    # Merge the duplicated dataframe with the aggreageted dataframe on 'Invoice #
    # 'Total_y' will be the correct total.
    merged_df = duplicated_invoices.merge(agg_total, on='Invoice #')
    merged_df['Total_x'] = np.where(merged_df['Total_x'], merged_df['Total_y'], merged_df['Total_y'])
    
    # Rename the 'Total_x' column to 'Total' before concatenating dataframes.
    # Finally drop the duplicated rows from the renamed_df.
    renamed_df = merged_df.drop(columns='Total_y').rename(columns={'Total_x':'Total'})
    renamed_df.drop_duplicates(subset=invoice_feature, keep='first', inplace=True)

    new_df = pd.concat([removed_duplicated, renamed_df], ignore_index=True)

    return new_df

In [17]:
plumbing = remove_duplicated_id(plumbing, 'Invoice #')

In [18]:
plumbing['Invoice #'].duplicated().sum()

0

In [19]:
plumbing.shape

(32737, 14)

### Location

In [20]:
plumbing['Location City'].unique()

array(['San Jose', 'Campbell', 'Santa Clara', 'Saratoga', 'Mountain View',
       'Cupertino', 'Stanford', 'Los Altos Hills', 'Los Gatos',
       'Sunnyvale', 'Milpitas', 'Los Altos', 'Monte Sereno', 'Gilroy',
       'Morgan Hill', 'Palo Alto', 'Fremont', 'Menlo Park', 'Palo Alto ',
       'East Palo Alto', 'Belmont', 'Redwood City', 'Burlingame',
       'Hayward', 'Newark', 'West Menlo Park', 'Santa Cruz', 'Woodside',
       'Atherton', 'Union City', 'Pittsburg', 'Stockton', 'Woodland',
       'American Canyon', 'Carmel-by-the-Sea', 'San Mateo', 'San Jose ',
       'Santa Clara ', 'Freedom', 'San Carlos', 'Alviso', 'san jose ',
       'Berkeley', 'Salinas', 'Campbell ', 'Solana Beach', 'Millbrae',
       'Portola Valley', 'Foster City', 'Irvine', 'Emerald Hills',
       'Los Gatos ', 'Hillsborough', 'san jose'], dtype=object)

In [21]:
plumbing['Location State'].unique()

array(['CA', 'AB'], dtype=object)

In [22]:
plumbing[plumbing['Location State']=='AB']

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date
31187,25341459,Residential,10/30/2022,0.0,1,1,724198,san jose,AB,95125,Area 1 - Drain,115.0,10/30/2022,10/30/2022


In [23]:
def clean_cityname(data, city_feature):
    """ This function corrects city names in the provided DataFrame.

    Args:
        data (DataFrame): The DataFrame containing the city name feature to be corrected.
        city_feature (str): The name of the city name feature in the DataFrame.

    Returns:
        pandas.Series: The cleaned city name feature."""
    
    sunnyvale = {'Sunnyvalve':'Sunnyvale','Sunnyvle':'Sunnyvale'}
    data[city_feature] = plumbing[city_feature].str.lstrip().str.rstrip().str.title()
    data[city_feature] = data[city_feature].replace(sunnyvale)

    return data[city_feature]

In [24]:
plumbing['Location State'] = 'CA'

In [25]:
plumbing['Location City'] = clean_cityname(plumbing, 'Location City')

In [26]:
def remove_low_city(data, city_feature):
    """ This function removes the cities with the invoice counts less than 50.

    Args:
        data (DataFrame): The DataFrame containing the city name feature.
        city_feature (str): The name of the city name feature in the DataFrame.

    Returns:
        DataFrame: The DataFrame with the low invoice counts cities removed."""

    city_count = data[city_feature].value_counts().reset_index()
    conditioned_city = list(city_count[city_count['count']<=50][city_feature])
    
    def check_city(city):
        """ This function checks if the city name is in the condition_city list and returns 1 if it is, and 0 if it's not.

        Args:
            city (str): The name of the city to check.

        Returns:
            1 (int): If the city name is in the list.
            0 (int): If the city name is not in the list."""
        
        for c in conditioned_city:
            if c in city:
                return 1
        return 0
    
    data['Low Invoice City'] = data[city_feature].apply(check_city)

    low_city_index = plumbing[plumbing['Low Invoice City']==1].index
    plumbing.drop(index=low_city_index, columns='Low Invoice City', inplace=True)

    return plumbing.reset_index(drop=True)

In [27]:
plumbing = remove_low_city(plumbing, 'Location City')
plumbing

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date
0,115098,Commercial,8/21/2022,180117.53,506,538,702069,San Jose,CA,95126,Area 1 - Drain,115.00,9/26/2020,9/26/2020
1,115098,Commercial,8/21/2022,180117.53,506,538,702176,San Jose,CA,95126,Area 1 - Plumbing,25.00,9/30/2020,9/30/2020
2,115098,Commercial,8/21/2022,180117.53,506,538,702194,San Jose,CA,95126,Area 1 - Drain,115.00,9/30/2020,9/30/2020
3,115098,Commercial,8/21/2022,180117.53,506,538,702217,San Jose,CA,95126,Area 1 - Drain,115.00,10/1/2020,10/1/2020
4,115098,Commercial,8/21/2022,180117.53,506,538,702220,San Jose,CA,95126,Area 1 - Drain,115.00,10/1/2020,10/1/2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32518,2727814,Residential,5/9/2022,855.25,5,5,703684,Cupertino,CA,95014,Area 1 - Drain,395.25,11/23/2020,2/13/2023
32519,260099,Residential,6/1/2020,3135.95,4,4,78469,Palo Alto,CA,94301,Area 1 - Plumbing,150.00,4/20/2020,3/1/2023
32520,11856257,Residential,5/19/2023,444.95,3,3,729881,San Jose,CA,95129,Area 1 - Drain,214.95,5/19/2023,5/19/2023
32521,4972306,Residential,6/9/2023,480.00,3,4,705355,San Jose,CA,95129,Area 1 - Drain,230.00,1/29/2021,2/13/2023


In [28]:
plumbing['Location City'].value_counts()

Location City
San Jose           16378
Sunnyvale           3458
Santa Clara         3092
Los Gatos           1815
Campbell            1362
Mountain View       1344
Cupertino           1267
Saratoga            1039
Palo Alto            794
Los Altos            644
Milpitas             589
Fremont              283
Menlo Park           153
Los Altos Hills      139
Redwood City         100
Monte Sereno          66
Name: count, dtype: int64

In [29]:
plumbing['Location Zip'].value_counts()

Location Zip
95126    1799
95050    1680
95125    1565
94087    1560
95008    1364
         ... 
94555       8
95002       4
94065       4
94027       2
94028       1
Name: count, Length: 64, dtype: int64

### Job Type

In [30]:
def extract_area(job_type_desc):
    """ This function extracts 'Area #' from the 'Job Type' feature.

    Args:
        job_type_desc (str): The job type description in the 'Job Type' feature.

    Returns:
        str or None: The extracted 'Area #' if found in the job type description,
        otherwise returns None."""
    
    match = re.search(r'Area (\d+)', job_type_desc)
    if match:
        return match.group(0)
    else:
        return None

In [31]:
plumbing['Area'] = plumbing['Job Type'].apply(extract_area)

In [32]:
plumbing['Area'].value_counts(dropna=False)

Area
Area 1    29369
Area 2     2088
None       1009
Area 3       57
Name: count, dtype: int64

In [33]:
def predict_area(data, x_feature, y_feature):
    """This function returns predictions using KNeighborsClassifier.

    Args:
        data (DataFrame): The DataFrame containing the data.
        x_feature (str): The name of the feature to be predicted.
        y_feature (str): The name of the feature used for prediction.

    Returns:
        array-like: An array containing the predicted values for the specified feature."""
    
    x = data[data[x_feature].notnull()][[y_feature]]
    y = data[data[x_feature].notnull()][[x_feature]]

    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(x,y)

    prediction = knn.predict(data[[y_feature]])

    return prediction

In [34]:
area_pred = predict_area(plumbing, 'Area', 'Location Zip')

c:\Users\nene0\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [35]:
def fill_missing_values(df, column_name, pred_list):
    """Fills missing values in the specified column of the DataFrame with values from the prediction list.

    Args:
        df (DataFrame): The DataFrame containing the column with missing values.
        column_name (str): The name of the column with missing values.
        pred_list (list): A list of values used to fill the missing values in the column.

    Returns:
        None """
    
    missing_indices = df[df[column_name].isnull()].index

    for i, index in enumerate(missing_indices):
        if i < len(pred_list):
            df.at[index, column_name] = pred_list[i]
        else:
            break

In [36]:
fill_missing_values(plumbing, 'Area', area_pred)

In [37]:
plumbing['Area'].value_counts()

Area
Area 1    30336
Area 2     2130
Area 3       57
Name: count, dtype: int64

In [38]:
def clean_jobtype(data, jobtype_feature):
    """Cleans the job type feature in the DataFrame by removing specific substrings, extra spaces, and leading/trailing spaces.

    Args:
        data (DataFrame): The DataFrame containing the job type feature to be cleaned.
        jobtype_feature (str): The name of the job type feature column in the DataFrame.

    Returns:
        pandas.Series: The cleaned job type feature as a pandas Series. """
    
    replace_dict = {'Area 1': '', 'Area 2': '', 'Area 3': '', '-': '', '  ':' '}

    data[jobtype_feature] = data[jobtype_feature].replace(replace_dict, regex=True)
    data[jobtype_feature] = data[jobtype_feature].str.rstrip()
    data[jobtype_feature] = data[jobtype_feature].str.lstrip()
    
    return data[jobtype_feature]

In [39]:
plumbing['Job Type'] = clean_jobtype(plumbing, 'Job Type')

In [40]:
plumbing['Job Type'].unique()

array(['Drain', 'Plumbing', 'Estimate', 'Water Heater',
       'Garbage Disposal bid price', 'Recall', 'No Hot Water',
       'Hydro Jetter 3"', 'Water Heater 75/100 galon',
       'Hydro truck 6" and larger',
       'Mainline stoppage with clean out access', 'Inspection',
       'Leak Locate', 'Faucet replacement', 'Pipe Leak',
       'Fixture Install', 'Slab Leak', 'Customer Supplied Parts',
       'Drain Mainline R/V NO ACCESS', 'French drain', 'Service Charge',
       'Holiday/Night Charge', 'Holiday/Night Rate', 'Waste and Overflow',
       'Pipe Bursting Technique Sewer line replacement', 'Repipe partial',
       'Garbage Disposal Bid price', 'Water Heater install 30/40/50',
       'MaintenanceHydro'], dtype=object)

### Date

In [41]:
plumbing.head(3)

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date,Area
0,115098,Commercial,8/21/2022,180117.53,506,538,702069,San Jose,CA,95126,Drain,115.0,9/26/2020,9/26/2020,Area 1
1,115098,Commercial,8/21/2022,180117.53,506,538,702176,San Jose,CA,95126,Plumbing,25.0,9/30/2020,9/30/2020,Area 1
2,115098,Commercial,8/21/2022,180117.53,506,538,702194,San Jose,CA,95126,Drain,115.0,9/30/2020,9/30/2020,Area 1


In [42]:
def clean_completion_date(data, date_feature, invoice_date):
    """Cleans the completion date feature in the DataFrame by handling specific date values and missing values.

    Args:
        data (DataFrame): The DataFrame containing the completion date feature to be cleaned.
        date_feature (str): The name of the completion date feature column in the DataFrame.
        invoice_date (str): The name of the invoice date feature column in the DataFrame.

    Returns:
        pandas.Series: The cleaned completion date feature as a pandas Series."""

    # Step 1: If completion date is '12/31/1899' then fill in the completion date as same as the invoice date.
    data[date_feature] = np.where(data[date_feature]=='12/31/1899', np.nan, data[date_feature])
    mask = data[date_feature].isna()
    data.loc[mask, date_feature] = data.loc[mask, invoice_date]

    # Step 2: If completion date is '1/11/2023', use the forward fill method to fill in missing values.
    data[date_feature] = np.where(data[date_feature]=='1/11/2023', np.nan, data[date_feature])
    data[date_feature] = data[date_feature].fillna(method='ffill')

    data[date_feature] = pd.to_datetime(data[date_feature])

    return data[date_feature]

In [43]:
def clean_invoice_date(data, date_feature):
    """Cleans the invoice date feature in the DataFrame by handling specific date values and missing values.

    Args:
        data (DataFrame): The DataFrame containing the invoice date feature to be cleaned.
        date_feature (str): The name of the invoice date feature column in the DataFrame.

    Returns:
        pandas.Series: The cleaned invoice date feature as a pandas Series."""
    
    data[date_feature] = np.where(data[date_feature]=='1/11/2023', np.nan, data[date_feature])
    data[date_feature] = data[date_feature].fillna(method='ffill')

    data[date_feature] = pd.to_datetime(data[date_feature])

    return data[date_feature]

In [44]:
plumbing['Completion Date'] = clean_completion_date(plumbing, 'Completion Date', 'Invoice Date')
plumbing['Invoice Date'] = clean_invoice_date(plumbing, 'Invoice Date')

In [45]:
month_name = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',
              9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

plumbing['Inv Month'] = plumbing['Invoice Date'].dt.month
plumbing['Inv Month Name'] = plumbing['Inv Month'].replace(month_name)

plumbing['Inv Year'] = plumbing['Invoice Date'].dt.year

plumbing['Invoice_Yr_Mo'] = plumbing['Inv Year'].astype('str') + '-' + plumbing['Inv Month Name']

plumbing['Weekday'] = plumbing['Invoice Date'].dt.day_name()

plumbing['Quarter'] = plumbing['Invoice Date'].dt.quarter

In [46]:
plumbing.shape

(32523, 21)

In [47]:
# Drop the 2020 April invoices as we do not have the full month of invoices.

plumbing.drop(index=plumbing[plumbing['Invoice_Yr_Mo']=='2020-Apr'].index, inplace=True)
plumbing.reset_index(drop=True, inplace=True)

In [48]:
plumbing.head()

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date,Area,Inv Month,Inv Month Name,Inv Year,Invoice_Yr_Mo,Weekday,Quarter
0,115098,Commercial,8/21/2022,180117.53,506,538,702069,San Jose,CA,95126,Drain,115.0,2020-09-26,2020-09-26,Area 1,9,Sep,2020,2020-Sep,Saturday,3
1,115098,Commercial,8/21/2022,180117.53,506,538,702176,San Jose,CA,95126,Plumbing,25.0,2020-09-30,2020-09-30,Area 1,9,Sep,2020,2020-Sep,Wednesday,3
2,115098,Commercial,8/21/2022,180117.53,506,538,702194,San Jose,CA,95126,Drain,115.0,2020-09-30,2020-09-30,Area 1,9,Sep,2020,2020-Sep,Wednesday,3
3,115098,Commercial,8/21/2022,180117.53,506,538,702217,San Jose,CA,95126,Drain,115.0,2020-10-01,2020-10-01,Area 1,10,Oct,2020,2020-Oct,Thursday,4
4,115098,Commercial,8/21/2022,180117.53,506,538,702220,San Jose,CA,95126,Drain,115.0,2020-10-01,2020-10-01,Area 1,10,Oct,2020,2020-Oct,Thursday,4


In [49]:
plumbing.describe()

,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Total,Completion Date,Invoice Date,Inv Month,Inv Year,Quarter
count,32299.000000,32299.000000,32299.000000,32299.000000,32299,32299,32299.000000,32299.000000,32299.000000
mean,18220.884815,58.953807,63.695842,345.952314,2022-02-03 12:06:50.613331712,2022-02-03 13:42:49.899996672,6.814267,2021.569708,2.602155
min,-16.000000,1.000000,1.000000,0.000000,2020-04-20 00:00:00,2020-05-01 00:00:00,1.000000,2020.000000,1.000000
25%,345.000000,2.000000,3.000000,115.000000,2021-04-05 00:00:00,2021-04-05 00:00:00,4.000000,2021.000000,2.000000
50%,1220.750000,6.000000,6.000000,115.000000,2022-01-20 00:00:00,2022-01-20 00:00:00,7.000000,2022.000000,3.000000
75%,7249.850000,20.000000,21.000000,223.375000,2022-11-19 00:00:00,2022-11-19 00:00:00,10.000000,2022.000000,4.000000
max,247901.860000,923.000000,982.000000,65350.000000,2024-01-31 00:00:00,2023-12-31 00:00:00,12.000000,2023.000000,4.000000
std,50007.091593,172.619792,185.626603,1249.452043,NaN,NaN,3.377753,1.030103,1.101619


### Customer's Last Service

In [50]:
plumbing['Last Job Completed'] = pd.to_datetime(plumbing['Last Job Completed'])

In [51]:
today = datetime.now()

plumbing['Last Service_Months'] = (today - plumbing['Last Job Completed']) / 30
plumbing['Last Service_Months'] = plumbing['Last Service_Months'].dt.days
plumbing['Last Service_Years'] = round(plumbing['Last Service_Months'] / 12, 2)

In [52]:
plumbing.head()

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date,Area,Inv Month,Inv Month Name,Inv Year,Invoice_Yr_Mo,Weekday,Quarter,Last Service_Months,Last Service_Years
0,115098,Commercial,2022-08-21,180117.53,506,538,702069,San Jose,CA,95126,Drain,115.0,2020-09-26,2020-09-26,Area 1,9,Sep,2020,2020-Sep,Saturday,3,19,1.58
1,115098,Commercial,2022-08-21,180117.53,506,538,702176,San Jose,CA,95126,Plumbing,25.0,2020-09-30,2020-09-30,Area 1,9,Sep,2020,2020-Sep,Wednesday,3,19,1.58
2,115098,Commercial,2022-08-21,180117.53,506,538,702194,San Jose,CA,95126,Drain,115.0,2020-09-30,2020-09-30,Area 1,9,Sep,2020,2020-Sep,Wednesday,3,19,1.58
3,115098,Commercial,2022-08-21,180117.53,506,538,702217,San Jose,CA,95126,Drain,115.0,2020-10-01,2020-10-01,Area 1,10,Oct,2020,2020-Oct,Thursday,4,19,1.58
4,115098,Commercial,2022-08-21,180117.53,506,538,702220,San Jose,CA,95126,Drain,115.0,2020-10-01,2020-10-01,Area 1,10,Oct,2020,2020-Oct,Thursday,4,19,1.58


In [53]:
plumbing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32299 entries, 0 to 32298
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Customer ID                 32299 non-null  object        
 1   Type                        32299 non-null  object        
 2   Last Job Completed          32299 non-null  datetime64[ns]
 3   Customers Lifetime Revenue  32299 non-null  float64       
 4   Lifetime Jobs Completed     32299 non-null  int64         
 5   Lifetime Invoices           32299 non-null  int64         
 6   Invoice #                   32299 non-null  object        
 7   Location City               32299 non-null  object        
 8   Location State              32299 non-null  object        
 9   Location Zip                32299 non-null  object        
 10  Job Type                    32299 non-null  object        
 11  Total                       32299 non-null  float64   

### Population

- This dataset is from: https://www.census.gov/

In [54]:
city_wide = pd.read_csv(r"C:\Users\nene0\Downloads\ca_city_pop.csv")

In [55]:
city_wide

,Fact,"Saratoga city, California","Cupertino city, California","Los Gatos town, California","Santa Clara city, California","Sunnyvale city, California","San Jose city, California","Milpitas city, California","Los Altos city, California","Palo Alto city, California","Mountain View city, California","Campbell city, California","Morgan Hill city, California","Redwood City city, California","Fremont city, California","Los Altos Hills town, California","Menlo Park city, California","Monte Sereno city, California","Carmel-By-The-Sea city, California","East Palo Alto city, California","Union City city, California","Belmont city, California","Newark city, California","Gilroy city, California"
0,"Population Estimates, July 1, 2022, (V2022)","29,903","57,856","32,402","126,930","153,091","971,233","77,738","30,424","66,010","81,059","42,286","44,973","80,512","223,871","8,168","32,295","3,479","3,220","28,383","67,039","26,773","47,312","58,005"
1,"Households, 2018-2022","10,798","20,744","12,855","47,434","58,646","324,842","24,395","10,887","26,064","34,516","17,064","14,575","29,993","74,995","3,084","11,661","1,270","1,578","7,998","20,860","10,811","14,805","16,969"


In [56]:
city_transposed = city_wide.T.reset_index()
city = city_transposed.drop(index=0)
city.columns = ['City', 'Population_Est','Num_Households']
replace_dict = {', California': '', ' city': '', ' town': '', ',': ''}

city['City'] = city['City'].replace(replace_dict, regex=True)

city['Population_Est'] = city['Population_Est'].replace(replace_dict, regex=True)
city['Num_Households'] = city['Num_Households'].replace(replace_dict, regex=True)

city['Population_Est'] = city['Population_Est'].astype('int64')
city['Num_Households'] = city['Num_Households'].astype('int64')

city

,City,Population_Est,Num_Households
1,Saratoga,29903,10798
2,Cupertino,57856,20744
3,Los Gatos,32402,12855
4,Santa Clara,126930,47434
5,Sunnyvale,153091,58646
6,San Jose,971233,324842
7,Milpitas,77738,24395
8,Los Altos,30424,10887
9,Palo Alto,66010,26064
10,Mountain View,81059,34516


In [57]:
residential_customer = plumbing[plumbing['Type']=='Residential']
city_grouped = residential_customer.groupby('Location City')[['Total','Invoice #']].agg({'Total':'sum','Invoice #':'count'}).reset_index().sort_values(by='Invoice #', ascending=False)
city_grouped.reset_index(drop=True, inplace=True)

In [58]:
location_service = city_grouped.merge(city, left_on='Location City', right_on='City')

location_service.drop(columns='City', inplace=True)

location_service['Normalized Services'] = location_service['Invoice #']/location_service['Num_Households']

location_service

,Location City,Total,Invoice #,Population_Est,Num_Households,Normalized Services
0,San Jose,3682814.42,12285,971233,324842,0.037818
1,Sunnyvale,916064.33,2733,153091,58646,0.046602
2,Santa Clara,811883.20,2509,126930,47434,0.052895
3,Los Gatos,437550.42,1480,32402,12855,0.115130
4,Cupertino,338852.20,1156,57856,20744,0.055727
5,Campbell,364736.95,1145,42286,17064,0.067100
6,Saratoga,312039.63,995,29903,10798,0.092147
7,Mountain View,420456.04,940,81059,34516,0.027234
8,Palo Alto,295991.88,687,66010,26064,0.026358
9,Los Altos,187565.02,544,30424,10887,0.049968


In [59]:
location_service.to_csv("city_household.csv", index=False)

### Weather

- This dataset is from: https://www.visualcrossing.com/

In [60]:
weather = pd.read_csv(r"C:\Users\nene0\OneDrive\바탕 화면\Python Learning\plumbing_project\sanjose_weather.csv",
                      parse_dates=['datetime'])

In [61]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1340 entries, 0 to 1339
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   datetime   1340 non-null   datetime64[ns]
 1   temp       1340 non-null   float64       
 2   precip     1340 non-null   float64       
 3   windgust   1340 non-null   float64       
 4   windspeed  1340 non-null   float64       
 5   Rain       1340 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 62.9 KB


In [62]:
weather.head()

,datetime,temp,precip,windgust,windspeed,Rain
0,2020-05-01,61.9,0.0,27.6,19.7,0
1,2020-05-02,63.3,0.0,23.9,17.2,0
2,2020-05-03,61.3,0.0,24.6,19.6,0
3,2020-05-04,61.2,0.0,23.9,16.1,0
4,2020-05-05,61.9,0.0,29.9,21.8,0


In [63]:
plumb_weather = plumbing.merge(weather, left_on='Invoice Date', right_on='datetime')
plumb_weather

,Customer ID,Type,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Invoice #,Location City,Location State,Location Zip,Job Type,Total,Completion Date,Invoice Date,Area,Inv Month,Inv Month Name,Inv Year,Invoice_Yr_Mo,Weekday,Quarter,Last Service_Months,Last Service_Years,datetime,temp,precip,windgust,windspeed,Rain
0,115098,Commercial,2022-08-21,180117.53,506,538,702069,San Jose,CA,95126,Drain,115.00,2020-09-26,2020-09-26,Area 1,9,Sep,2020,2020-Sep,Saturday,3,19,1.58,2020-09-26,69.4,0.00,27.7,15.0,0
1,680733,Residential,2024-02-06,2085.75,19,20,702062,San Jose,CA,95112,Drain,115.00,2020-09-26,2020-09-26,Area 1,9,Sep,2020,2020-Sep,Saturday,3,2,0.17,2020-09-26,69.4,0.00,27.7,15.0,0
2,680733,Residential,2024-02-06,2085.75,19,20,702077,San Jose,CA,95112,Plumbing,0.00,2020-09-26,2020-09-26,Area 1,9,Sep,2020,2020-Sep,Saturday,3,2,0.17,2020-09-26,69.4,0.00,27.7,15.0,0
3,612483,Residential,2021-04-06,2309.40,9,9,702064,Santa Clara,CA,95050,Plumbing,115.00,2020-09-26,2020-09-26,Area 1,9,Sep,2020,2020-Sep,Saturday,3,36,3.00,2020-09-26,69.4,0.00,27.7,15.0,0
4,1738753,Residential,2023-04-28,14890.00,8,9,702018,San Jose,CA,95116,Drain,115.00,2020-09-26,2020-09-26,Area 1,9,Sep,2020,2020-Sep,Saturday,3,11,0.92,2020-09-26,69.4,0.00,27.7,15.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32294,27311873,Residential,2023-01-09,0.00,2,2,725786,Los Altos,CA,94024,Drain,0.00,2023-01-03,2023-01-03,Area 1,1,Jan,2023,2023-Jan,Tuesday,1,15,1.25,2023-01-03,53.0,0.00,21.9,10.2,0
32295,1825427,Residential,2022-01-02,1765.00,2,2,715397,San Jose,CA,95129,Plumbing,1650.00,2022-01-02,2022-01-02,Area 1,1,Jan,2022,2022-Jan,Sunday,1,27,2.25,2022-01-02,44.0,0.00,21.0,9.1,0
32296,27200150,Residential,2022-12-26,0.00,1,1,725720,San Jose,CA,95110,Plumbing,0.00,2022-12-26,2022-12-26,Area 1,12,Dec,2022,2022-Dec,Monday,4,15,1.25,2022-12-26,52.6,0.06,17.2,16.8,1
32297,7160966,Commercial,2024-02-02,247901.86,923,982,721084,Redwood City,CA,94061,Drain,439.95,2022-07-03,2022-07-04,Area 2,7,Jul,2022,2022-Jul,Monday,3,2,0.17,2022-07-04,68.4,0.00,20.8,19.6,1


In [64]:
plumb_weather.drop(columns='datetime', inplace=True)

In [65]:
plumb_weather.describe()

,Last Job Completed,Customers Lifetime Revenue,Lifetime Jobs Completed,Lifetime Invoices,Total,Completion Date,Invoice Date,Inv Month,Inv Year,Quarter,Last Service_Months,Last Service_Years,temp,precip,windgust,windspeed,Rain
count,32299,32299.000000,32299.000000,32299.000000,32299.000000,32299,32299,32299.000000,32299.000000,32299.000000,32299.000000,32299.000000,32299.000000,32299.000000,32299.000000,32299.000000,32299.000000
mean,2023-01-08 05:22:04.239140608,18220.884815,58.953807,63.695842,345.952314,2022-02-03 12:06:50.613331712,2022-02-03 13:42:49.899997184,6.814267,2021.569708,2.602155,14.642806,1.220441,60.811270,0.023463,20.603242,14.682916,0.168612
min,2020-05-01 00:00:00,-16.000000,1.000000,1.000000,0.000000,2020-04-20 00:00:00,2020-05-01 00:00:00,1.000000,2020.000000,1.000000,1.000000,0.080000,43.300000,0.000000,4.700000,4.600000,0.000000
25%,2022-05-02 00:00:00,345.000000,2.000000,3.000000,115.000000,2021-04-05 00:00:00,2021-04-05 00:00:00,4.000000,2021.000000,2.000000,4.000000,0.330000,54.000000,0.000000,15.700000,11.500000,0.000000
50%,2023-05-19 00:00:00,1220.750000,6.000000,6.000000,115.000000,2022-01-20 00:00:00,2022-01-20 00:00:00,7.000000,2022.000000,3.000000,10.000000,0.830000,61.200000,0.000000,19.700000,14.800000,0.000000
75%,2023-12-05 00:00:00,7249.850000,20.000000,21.000000,223.375000,2022-11-19 00:00:00,2022-11-19 00:00:00,10.000000,2022.000000,4.000000,23.000000,1.920000,67.100000,0.000000,25.300000,17.200000,0.000000
max,2024-02-12 00:00:00,247901.860000,923.000000,982.000000,65350.000000,2024-01-31 00:00:00,2023-12-31 00:00:00,12.000000,2023.000000,4.000000,47.000000,3.920000,88.500000,1.227000,56.600000,38.400000,1.000000
std,NaN,50007.091593,172.619792,185.626603,1249.452043,NaN,NaN,3.377753,1.030103,1.101619,12.687899,1.057164,8.120591,0.107703,7.401346,4.271168,0.374415


In [66]:
plumb_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32299 entries, 0 to 32298
Data columns (total 28 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Customer ID                 32299 non-null  object        
 1   Type                        32299 non-null  object        
 2   Last Job Completed          32299 non-null  datetime64[ns]
 3   Customers Lifetime Revenue  32299 non-null  float64       
 4   Lifetime Jobs Completed     32299 non-null  int64         
 5   Lifetime Invoices           32299 non-null  int64         
 6   Invoice #                   32299 non-null  object        
 7   Location City               32299 non-null  object        
 8   Location State              32299 non-null  object        
 9   Location Zip                32299 non-null  object        
 10  Job Type                    32299 non-null  object        
 11  Total                       32299 non-null  float64   

### Save & Separate Data

In [67]:
residential = plumb_weather[plumb_weather['Type']=='Residential']
residential = residential.reset_index(drop=True)
residential.shape

(25498, 28)

In [68]:
commercial = plumb_weather[plumb_weather['Type']=='Commercial']
commercial = commercial.reset_index(drop=True)
commercial.shape

(6801, 28)

In [69]:
residential.to_csv('Residential_Plumbing.csv', index=False)
commercial.to_csv('Commercial_Plumbing.csv', index=False)

In [74]:
plumbing.to_csv('Cleaned_Data.csv', index=False)

In [73]:
plumbing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32299 entries, 0 to 32298
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Customer ID                 32299 non-null  object        
 1   Type                        32299 non-null  object        
 2   Last Job Completed          32299 non-null  datetime64[ns]
 3   Customers Lifetime Revenue  32299 non-null  float64       
 4   Lifetime Jobs Completed     32299 non-null  int64         
 5   Lifetime Invoices           32299 non-null  int64         
 6   Invoice #                   32299 non-null  object        
 7   Location City               32299 non-null  object        
 8   Location State              32299 non-null  object        
 9   Location Zip                32299 non-null  object        
 10  Job Type                    32299 non-null  object        
 11  Total                       32299 non-null  float64   